In [1]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
import time
from threading import Thread
from threading import Event
from GammaClasses import PE
from GammaClasses import Scheduler
from GammaClasses import FiberCache

In [2]:
I = 10
K = 10
J = 10
NUM_INTS = 1000
NUM_PES = 32
RADIX = 3

gen = np.random.default_rng()
data1 = gen.integers(1,10,NUM_INTS)
row1 = gen.integers(0,I,NUM_INTS)
col1 = gen.integers(0,K,NUM_INTS)

data2 = gen.integers(1,10,NUM_INTS)
row2 = gen.integers(0,K,NUM_INTS)
col2 = gen.integers(0,J,NUM_INTS)
i1 = csr_matrix(coo_matrix((data1, (row1, col1)), shape=(I, K)))
i2 = csr_matrix(coo_matrix((data2, (row2, col2)), shape=(K, J)))
print(i1.toarray())
print(i2.toarray())

[[14 12  6  8  8  5  8  1  0  7]
 [ 0  0  9  0 14 24  9  7  0  0]
 [ 0  0  9  8  9  3  7  0  8  8]
 [ 0 25  0 21  3 16  3  2 14  0]
 [ 3  0 12 21  7  7  9  4  7  1]
 [ 3  0  5  0  0  3  0  0  0  0]
 [13  0  0  0  2  0 13 11  0  4]
 [11 16  0  0  0  0  6  0  3  0]
 [ 0  0  0  5 18  3  7  4  4  0]
 [ 2  3  0 12  8  5 17  9  5  0]]
[[ 0  1  0  0  5  0  5 11  3  0]
 [ 6  8  0  3  6  0  0  7  2  0]
 [ 8  7 12  0  8 16  5  0  6  0]
 [ 0  1 24  0  8  5  5  0  1  0]
 [ 4 14  2 15 23 15  3  7 13  6]
 [ 0  0  0  5  4  9  8 13  4 19]
 [ 0  0  3  5  0  1  1  6  0  7]
 [ 0  6  0  3  7 11  0  4  0  9]
 [ 7  0  0  7 10  1  0 19  8  8]
 [11  6  0  8  9 14  0  9  0  5]]


In [3]:
PEArray = []
for x in range(NUM_PES):
    PEArray.append(PE(RADIX))

S = Scheduler(PEArray,i1,RADIX)

for x in S.jobs:
    print(x)

FC = FiberCache()
for x in range(NUM_PES):
    PEArray[x].connect(S,FC)


0 0 9
1 9 14
2 14 21
3 21 28
4 28 37
5 37 40
6 40 45
7 45 49
8 49 55
9 55 63
(True, False, 0, 1, [2, 3, 4], [1, 1, 1])
(False, True, 0, 2, array([0, 1, 2]), array([14, 12,  6], dtype=int64))
(False, True, 0, 3, array([3, 4, 5]), array([8, 8, 5], dtype=int64))
(False, True, 0, 4, array([6, 7, 9]), array([8, 1, 7], dtype=int64))
(True, False, 1, 5, [6, 7, 8], [1, 1, 1])
(False, True, 1, 6, array([2, 4, 5]), array([ 9, 14, 24], dtype=int64))
(False, True, 1, 7, array([6]), array([9], dtype=int64))
(False, True, 1, 8, array([7]), array([7], dtype=int64))
(True, False, 2, 9, [10, 11, 12], [1, 1, 1])
(False, True, 2, 10, array([2, 3, 4]), array([9, 8, 9], dtype=int64))
(False, True, 2, 11, array([5, 6, 8]), array([3, 7, 8], dtype=int64))
(False, True, 2, 12, array([9]), array([8], dtype=int64))
(True, False, 3, 13, [14, 15, 16], [1, 1, 1])
(False, True, 3, 14, array([1, 3, 4]), array([25, 21,  3], dtype=int64))
(False, True, 3, 15, array([5, 6, 7]), array([16,  3,  2], dtype=int64))
(False, 

In [4]:
for x in range(NUM_PES):
    print(PEArray[x].nextFiberSet)

(True, False, 0, 1, [2, 3, 4], [1, 1, 1])
(False, True, 0, 2, array([0, 1, 2]), array([14, 12,  6], dtype=int64))
(False, True, 0, 3, array([3, 4, 5]), array([8, 8, 5], dtype=int64))
(False, True, 0, 4, array([6, 7, 9]), array([8, 1, 7], dtype=int64))
(True, False, 1, 5, [6, 7, 8], [1, 1, 1])
(False, True, 1, 6, array([2, 4, 5]), array([ 9, 14, 24], dtype=int64))
(False, True, 1, 7, array([6]), array([9], dtype=int64))
(False, True, 1, 8, array([7]), array([7], dtype=int64))
(True, False, 2, 9, [10, 11, 12], [1, 1, 1])
(False, True, 2, 10, array([2, 3, 4]), array([9, 8, 9], dtype=int64))
(False, True, 2, 11, array([5, 6, 8]), array([3, 7, 8], dtype=int64))
(False, True, 2, 12, array([9]), array([8], dtype=int64))
(True, False, 3, 13, [14, 15, 16], [1, 1, 1])
(False, True, 3, 14, array([1, 3, 4]), array([25, 21,  3], dtype=int64))
(False, True, 3, 15, array([5, 6, 7]), array([16,  3,  2], dtype=int64))
(False, True, 3, 16, array([8]), array([14], dtype=int64))
(True, False, 4, 17, [18, 